In [1]:
import numpy as np
import sys
sys.path.append('..')

import os
import re
import json
import numpy as np
from utils.utils import *
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import trange
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import dill
import matplotlib.pyplot as plt
from tqdm import tqdm
# print(torch.__version__)
# print(torch.version.cuda)
# use_gpu = torch.cuda.is_available()
# use_gpu
sns.set_theme(style="darkgrid")
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
target = "5_4"
# target = "5_5"
# target = "p2"

获得同时经过两个区域的轨迹的id以及在各自区域对应的索引

In [2]:
ids_5_5 = np.load("/root/multiscale/tra_id_listfutian5_5_10685.npy",allow_pickle=True).tolist()
ids_5_4 = np.load("/root/multiscale/tra_id_listfutian5_4_39895.npy",allow_pickle=True).tolist()
ids_p2 = np.load("/root/multiscale/tra_id_futian_p2_list0613.npy",allow_pickle=True).tolist()

ids_common = np.load("/root/multiscale/common_tra_id_listfutian6364.npy",allow_pickle=True).tolist()
index_list_5_4=[]
index_list_5_5=[]
index_list_p2=[]
for id in ids_common:
    index_list_5_4.append(  ids_5_4.index(id))
    index_list_5_5.append(  ids_5_5.index(id))
    if id in ids_p2:
        index_list_p2.append(  ids_p2.index(id))
print(len(ids_common),len(index_list_5_4),len(index_list_5_5),len(index_list_p2))


6364 6364 6364 4000


数据初始化

In [3]:
# 设置基础参数和文件路径
l = 10000 # 边的数量
m = 30000 # p空间的数量
n = 300 # 轨迹数据集的大小
# 超参数
mu=10
la=10.0/n
if target=="5_4":
    raw_tras_name = "tra_datasetfutian5_4_39895"
if target=="5_5":
    raw_tras_name = "tra_datasetfutian5_5_10685"
if target=="p2":
    raw_tras_name = "tra_dataset_futian_p2_40000613"


raw_tras_sz = load_tras("/root/multiscale/"+raw_tras_name+".npy")
print("len of raw tras when loading:",len(raw_tras_sz))
tras,tra_index_list = filter_raw_num_of_edge_6(raw_tras_sz)
# np.save('tra_index_list.npy',np.array(tra_index_list,dtype=object))
if target=="5_4":
    tras = [tras[index] for index in index_list_5_4]
if target=="5_5":
    tras = [tras[index] for index in index_list_5_5]
if target=="p2":
    tras = [tras[index] for index in index_list_p2]

tras = tras[:4000] 
print("len of raw tras after filter:",len(tras))

len of raw tras when loading: 39895
len of raw tras after filter edges more than 6: 39895
the first trajectory:
[(7902654987, 6020564585), (6020564585, 4138689909), (4138689909, 9660848465), (9660848465, 9660848466), (9660848466, 7868781069), (7868781069, 9660848467), (9660848467, 9660848480), (9660848480, 9660848481), (9660848481, 9660848469), (9660848469, 9660848464), (9660848464, 9660848463), (9660848463, 6020564517), (6020564517, 9660848462), (9660848462, 6020564510), (6020564510, 9608302600), (9608302600, 2269583663), (2269583663, 2269583676)]
split tras and tras_test
tras chosen: 39895
len of raw tras after filter: 4000


In [4]:
# 我们需要获得所有的edge并且编号；并且获得所有的t与e的对应关系 这样方便下一步计算矩阵，也就是得到edge_list and path_list
# 先计算一下e和p的大小
edge_list = []
path_list = []
path_list_cnt_ = []
tra_index = 0
def generate_all_p_(t,min,max):
    res=[]
    for i in range(len(t)):
        for j in range(i+1,len(t)+1):
            if ((j-i)>min) and ((j-i)<max):
                res.append(t[i:j])
    return res
# path_list_dict={}#记录不同的pathlet
path_list_dict_cnt={}#记录不同的pathlet对应的被经过的次数
for i in range(len(tras)):
    tra = tras[i]
    for edge in tra:
        edge_list.append(str(edge).replace(" ","")) 
    path_min=12
    path_max=30
    if target=="p2":
        path_min=1
        path_max=30
    for path in generate_all_p_(tra,min=path_min,max=path_max):
        path_len = len(path)
        # if path_len>10:
        #     print(path_len)
        path_str = str(path).replace(" ","")
        path_list.append(path_str)
        if path_str not in path_list_dict_cnt.keys():
            path_list_dict_cnt[path_str]=[1,0]
        else:
            path_list_dict_cnt[path_str][0]+=1
l,m,n = len(edge_list)+10,len(path_list)+10,len(tras)
print("len(edge_list):",l,"len(path_list):",m,"len(tras):",n)
edge_list = list(set(edge_list))           
path_list = list(path_list_dict_cnt.keys())
l,m,n = len(edge_list)+10,len(path_list)+10,len(tras)
print("len(edge_list):",l,"len(path_list):",m,"len(tras):",n)
# 这一次是根据pathlet的frequency来过滤一部分不太常用的path
filter_num=1
if target=="p2":
    filter_num=0
old_path_list = path_list
path_list = []
path_index=0
for i in range(len(old_path_list)):
    if path_list_dict_cnt[old_path_list[i]][0]>filter_num:
        path_list.append(old_path_list[i])
        path_list_dict_cnt[old_path_list[i]][1]=path_index
        path_index+=1
# for i in trange(len(tras)):
#     tra = tras[i]
#     path_list.append(str(tra).replace(" ",""))
l,m,n = len(edge_list)+10,len(path_list)+10,len(tras)
print("len(edge_list):",l,"len(path_list):",m,"len(tras):",n)

# # 存储pathlet_list
np.save('path_list'+raw_tras_name+'.npy',np.array(path_list,dtype=object))   # 保存为.npy格式
# np.save('edge_list.npy',np.array(edge_list,dtype=object))   # 保存为.npy格式

len(edge_list): 104466 len(path_list): 571195 len(tras): 4000
len(edge_list): 3866 len(path_list): 271457 len(tras): 4000
len(edge_list): 3866 len(path_list): 45086 len(tras): 4000


In [5]:
# 计算三个矩阵19868742712
D = np.zeros((l, m)) #记录e与p之间关系
R = np.zeros(( m,n)) #记录p与t之间关系
T = np.zeros((l, n)) #记录e和t之间关系
D_tensor=torch.zeros(len(edge_list)+10,len(path_list)+10)#.cuda()
computed_path = []

tra_index = 0
for i in trange(len(tras)):
    tra = tras[i]
    for edge in tra:
        T[edge_list.index(str(edge).replace(" ",""))][i] = 1
    # path_index = path_list.index(str(tra).replace(" ",""))
    # R[path_index][i]=1
from tqdm import tqdm
for path in tqdm(path_list):
    path_index = path_list.index(path)
    edge_list_of_temp_pathlet= path_str2list(path)
    for edge in edge_list_of_temp_pathlet:
        if edge in edge_list:
            D_tensor[edge_list.index(edge)][path_index] = 1

D_tensor=D_tensor#.cuda()
T_tensor=torch.from_numpy(T[:len(edge_list)+10])#.cuda()
print('x.dtype: ',D_tensor.dtype)  # x的具体类型
R_tensor_tejie=torch.from_numpy(R[:len(path_list)+10])
# R_tensor=torch.rand(len(path_list)+10,n).double()
# D=np.load("./D.npy",allow_pickle=True)
# T=np.load("./T.npy",allow_pickle=True)
# R=np.load("./R.npy",allow_pickle=True)
# # 保存数据为csv，供matlab使用
# numpy.savetxt("D_tensor_148.csv", D_tensor.cpu().detach().numpy(), delimiter=',')
# numpy.savetxt("T_tensor_148.csv", T_tensor[:,0:30].cpu().detach().numpy(), delimiter=',')

  0%|          | 0/4000 [00:00<?, ?it/s]

100%|██████████| 45076/45076 [01:49<00:00, 412.51it/s]

x.dtype:  torch.float32


# 训练开始

In [6]:
import copy

initial_R = [
    # torch.rand(len(path_list)+10,n).double(), # random初始化
torch.from_numpy(R[:len(path_list)+10]), #全不分割初始化
torch.zeros( m,n) #全0初始化
]

LR = 0.005
epoch_num =500
final_loss = []
R_list = []
value_distribution_list = []
R_tensor_nonzero_ratio = []
subset=0
num_of_subset=10000
gamma_value=0.96


In [7]:

mu_values = [0,0,0.81,0.2]
# mu_values = [0,0,1.51,2]
la_list = [0,0.1,0.3,0.7,1,5]
la_list=[0.1]
for la in la_list:
    loss_his,R_tensor,muvalue = R_train(initial_R[0],D_tensor.float().cuda(),T_tensor.float().cuda(),
subset,num_of_subset,la,mu_values[0],mu_values[1],mu_values[2],mu_values[3],epoch_num,LR,gamma_value)
    final_loss.append(loss_his)
    
    report_performance(D_tensor.cuda(),R_tensor.cuda(),T_tensor.cuda())
    R_list.append(copy.deepcopy(R_tensor.cpu().detach().numpy()))
# 是否只使用一部分的字典来重构轨迹数据集
# 如果需要的话 需要改subset的值为1 同时注释 for control_lambda in control_lambda_list:部分
#  这样的得到的结果就是当保留num_of_subset这么多pathlets的时候的性能
# subset=1
# num_of_subset = [100,200,300,500,1000,1500,2000,3000,4000]
# # if subset==1:
# loss_subset_record = []
# for i in num_of_subset:
#     p_picked_index = list(pathlet_dictionary["index"]) #这是被选中到字典中的p的index
#     p_picked_index_subset = p_picked_index[:i]
#     D_tensor_subset=(D_tensor[:len(edge_list)+10,:len(path_list)+10]).cuda()
#     # D_tensor_test_zeros = copy.deepcopy(D_tensor)
#     for j in range(D_tensor_subset.shape[1]):
#         if j  not in p_picked_index_subset:
#             D_tensor_subset[:,j]=0
#     control_lambda=0.15*len(tras)
#     la = 0.1#control_lambda/len(tras)
#     loss_his_subset,R_tensor_subset,muvalue = R_train(initial_R[0],D_tensor_subset.float().cuda(),T_tensor.float().cuda()
#                                               ,subset,i,la,mu_values[0],mu_values[1],mu_values[2],mu_values[3],epoch_num,LR,gamma_value)
#     loss_subset_record.append(loss_his_subset)

# diff datasets 
# diff lambda
# diff theta   

-------------------------------------------------lambda_value= 0.1


100%|██████████| 500/500 [03:41<00:00,  2.26it/s]


dict_size 1118.699951171875
representation_cost 7157.2998046875
avg representation_cost 1.789324951171875
num of all edges: 104087.0
num of uncover edges: 6052.47
num of overlap edges: 18699.705
uncover ratio: 0.0581
overlap ratio: 0.18


# 开始随机rounding，输出的结果为R_tensor_rounding，并且获得字典

In [8]:
import copy
# R_list.append(copy.deepcopy(R_tensor.cpu().detach().numpy()))
import math
# 随机rounding
from scipy.stats import bernoulli
# wuwu_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5]
wuwu_list = [0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7]
wuwu_list =[1]
# for wuwu in wuwu_list:
cnt=0
pathlet_dictionary_list=[]
rounding_loss_under_las = []
repeated_rounding_num=1
rounded_R_tensor_list = [] 
iterate_cnt=0
for R_data_load in R_list[:]:
    print(iterate_cnt)
    iterate_cnt+=1
    rounding_loss=[[],[],[],[],[],[],[]]
    print("rounding procedure start")
    if 1:
        wuwu=1
        wuwu=math.log(2*torch.norm(T_tensor,1).cpu().data.numpy().round(1))*0.2
        R_data_star = copy.deepcopy(R_data_load)
        # R_data = copy.deepcopy(R_data_load)
        R_data = np.zeros((R_data_star.shape[0],R_data_star.shape[1],repeated_rounding_num))
        for i in trange(R_data_star.shape[0]):
            if R_data_star[i].sum()>0.1:
                for j in range(R_data_star.shape[1]):
                    if(R_data_star[i][j]>0.03):
                        R_data[i][j][:] = bernoulli.rvs(size=repeated_rounding_num,p=min(1,wuwu*R_data_star[i][j]))
                    else:
                        R_data[i][j]=0
        # .data = R_data_load
    print("rounding procedure end")
    print("loss compute start")
    R_data_tensor = torch.from_numpy(R_data)
    relu = torch.nn.ReLU().cuda()
    if_subset=0
    diff_norm_num=2
    for ite in range(repeated_rounding_num):
        R_tensor_rounding = R_data_tensor[:,:,ite].cuda().float()
        # print("-------------------------------")
        # print("wuwu:",wuwu)
        report_performance(D_tensor.cuda(),R_tensor_rounding.cuda(),T_tensor.cuda())
        # rounding_loss[0].append((a+b+c+d+e+f).cpu().data.numpy().round(1))
        # rounding_loss[1].append(a.cpu().data.numpy().round(1))
        # rounding_loss[2].append(b.cpu().data.numpy().round(1))
        # rounding_loss[3].append(c.cpu().data.numpy().round(1))
        # rounding_loss[4].append(d.cpu().data.numpy().round(1))
        # rounding_loss[5].append(e.cpu().data.numpy().round(1))
        # rounding_loss[6].append(f.cpu().data.numpy().round(1))
    # rounding_loss_under_las.append(rounding_loss)
    print("loss compute end")
    print("compute dictionary start")
# if 1:
    # 根据R_tensor_rounding计算每一个pathlet被用到的次数，并查询得到每一个path的长度
    path_list_len = len(path_list)
    path_list_cnt = np.zeros((path_list_len+10,3))
    R_size = R_tensor_rounding.size()
    pathlet_cnt = torch.sum(R_tensor_rounding, dim=1).cpu().numpy().round(1)
    for i in trange(R_size[0]):
        path_list_cnt[i][0]=i
        path_list_cnt[i][1]=pathlet_cnt[i]
        if i <path_list_len:
            path_list_cnt[i][2]=len(path_list[i])

    # 获得字典
    path_list_cnt_df = pd.DataFrame(path_list_cnt,columns=["index","cnt",'len'])
    pathlet_dictionary = path_list_cnt_df[path_list_cnt_df["cnt"]>0].sort_values(by=['cnt'],ascending = False)
     
    pathlet_dictionary_list.append(pathlet_dictionary)
    print("compute dictionary end")
    rounded_R_tensor_list.append(R_data_tensor)



0
rounding procedure start


100%|██████████| 45086/45086 [01:23<00:00, 537.47it/s] 


rounding procedure end
loss compute start
dict_size 4666.0
representation_cost 14762.0
avg representation_cost 3.6905
num of all edges: 104087.0
num of uncover edges: 10980.0
num of overlap edges: 145747.0
uncover ratio: 0.1055
overlap ratio: 1.4
loss compute end
compute dictionary start


100%|██████████| 45086/45086 [00:00<00:00, 520908.55it/s]

compute dictionary end


In [9]:
for i in range(len(rounded_R_tensor_list)):
    target_save_R_tensor = rounded_R_tensor_list[i]
    if target=="p2":
        torch.save(target_save_R_tensor, "./R_tensor_rounded_noself_p20613"+raw_tras_name+str(i)+"_.pt")
    else:
        torch.save(target_save_R_tensor, "./R_tensor_rounded_noself"+raw_tras_name+str(i)+"_.pt")

